Run the config file to authenticate script and query Key Vault

Second script. Run after 'Query_Activites'

In [ ]:
%run config

Dependencies 

In [ ]:
from pyspark.sql.functions import * 
import pandas as pd

### API Query to get more specific details for each activity, need to pass each activity off individually 

Get full activity dataset from what is written in storage, should be all activites

In [ ]:

full_activity_dataset = spark.read.format("delta").load(historical_activity_id_path)

In [ ]:
full_activity_dataset.show()

In [ ]:
full_activity_dataset.select('ingested_at').distinct().show()

Ggrab all of the disinct activity IDs 

In [ ]:
full_activity_ids = full_activity_dataset.select('activity_ids').distinct().rdd.flatMap(lambda x: x).collect()

Take unique activity ids, and extract all of the segments associated with those activities

In [ ]:
def query_segments(activity_ids):
    """Gets all segment_ids for each activity_id submitted
    Returns distinct values"""
    df = pd.DataFrame()
    activity_id_list =[]
    segment_id_list =[]
    for id in activity_ids:
        activity_id_urls = ("{}{}?include_all_efforts= True").format("https://www.strava.com/api/v3/activities/",id)
        header = {'Authorization': 'Bearer ' + access_token}
        param = {'per_page': 200, 'page': 1}
        my_activity = requests.get(activity_id_urls, headers=header, params=param).json()

        segment_effort_count =  len(my_activity['segment_efforts'])
        count = 0
        while count < segment_effort_count:

            activity_id = my_activity['segment_efforts'][count]['activity']['id']
            segment_id = my_activity['segment_efforts'][count]['id']
            activity_id_list.append(activity_id)
            segment_id_list.append(segment_id)
                  
            columns = ['segment_id', 'activity_id']
            extracted_data = [segment_id_list, activity_id_list]
            segment_df = pd.DataFrame.from_dict(dict(zip(columns, extracted_data)))

            df = pd.concat([df, segment_df])

            #df= df.append(segment_df)
            count += 1

    #convert pandas df to spark
        
    segment_spark_df = spark.createDataFrame(df)

    #drop duplicate entries
    segment_spark_df = segment_spark_df.dropDuplicates()

    segment_spark_df = segment_spark_df.select(concat(segment_spark_df.segment_id,segment_spark_df.activity_id).alias("Activity_Segment_JointID"), 'segment_id','activity_id')

    segment_spark_df = segment_spark_df.withColumn("ingest_file_name", lit("segment_efforts_ids")) \
                                .withColumn("ingested_at", lit(current_timestamp()))

    return segment_spark_df

Need to compare activites already stored with segments as there are limits for strava API

Initially will go 99 request, but might need to reduce that to save requests for segments

Query all segments from all activities 

In [ ]:
#testing segment query on small list of activities
test_id_list = full_activity_ids[:5]
segment_id_df = query_segments(test_id_list)

In [ ]:
segment_id_df.show(5)

Not all activities register segments, will need to append into final DF that is writtent to storage with placerholder values so as not to keep querying them

In [ ]:
segment_id_df.select("activity_id").distinct().show()

In [ ]:
try:
    #Query path, see if there are any activities with their associated segments written to storage
    segments_in_storage = spark.read.format("delta").load(segment_effort_path)
except:
    #if that errors, meaning first time running the script
    #Write the first 99 activites to storage, will need to specificy sort order 
    top_99_activity_ids = 
    write_dataframe_to_storage(historical_df_to_write,storagepath, "mergeSchema", "append" )


Need to limit to 100 request as the api throws an errors after

In [ ]:
write_dataframe_to_storage(segment_id_df,segment_effort_path, "overwriteSchema","overwrite" )

In [ ]:
segments_in_storage = spark.read.format("delta").load(segment_effort_path)

In [ ]:
#need to get distinct activity_ids and run them through the segment
#we make 1 api query in the first script, so gonna be allowed 99 with this run 

In [ ]:
#get all of the activity_ids, limit to 15 as thats how many we can run in a single query
#also need to query the activities we have written to segment storage, so as not to repeat 
#Strava API usage is limited on a per-application basis using both a 15-minute and daily request limit. The default rate limit allows 100 requests every 15 minutes, 
# with up to 1,000 requests per day.

#compare current activites vs what is written, 
#going to need to write some try and excepts for expecting these values in return 